# 解答 1: 基礎文檔處理實作
## Solution 1: Basic Document Processing Implementation

> **說明**: 練習 1 的完整實作解答與詳細註解

---

## 🎯 解答概覽

本解答提供了完整的實作代碼，包含：
1. **詳細的實作邏輯** - 每個方法的完整實現
2. **程式碼註解** - 解釋關鍵設計決策
3. **最佳實務** - 錯誤處理和邊界條件
4. **測試與驗證** - 完整的測試流程

---

## 🔧 環境設定

In [ ]:
import os
import re
import json
import hashlib
import numpy as np
from pathlib import Path
from datetime import datetime
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
import pandas as pd

# 設定專案路徑
PROJECT_ROOT = Path('/home/os-sunnie.gd.weng/python_workstation/side-project/RAG/data_governance/kms_governance')
SOLUTION_DIR = PROJECT_ROOT / 'notebooks' / '01_document_ingestion' / 'solutions'
EXERCISE_DIR = PROJECT_ROOT / 'notebooks' / '01_document_ingestion' / 'exercises'

print("✅ 環境設定完成")

## 📋 資料結構定義

In [ ]:
@dataclass
class SimpleMetadata:
    """簡化的文檔元資料"""
    doc_id: str
    filename: str
    file_size: int
    created_time: datetime
    doc_type: str
    word_count: int
    language: str = 'en'
    
    def to_dict(self) -> Dict:
        """轉換為字典格式"""
        return {
            'doc_id': self.doc_id,
            'filename': self.filename,
            'file_size': self.file_size,
            'created_time': self.created_time.isoformat(),
            'doc_type': self.doc_type,
            'word_count': self.word_count,
            'language': self.language
        }

@dataclass
class SimpleChunk:
    """簡化的分塊資料"""
    chunk_id: str
    doc_id: str
    content: str
    chunk_index: int
    word_count: int
    
    def to_dict(self) -> Dict:
        """轉換為字典格式"""
        return {
            'chunk_id': self.chunk_id,
            'doc_id': self.doc_id,
            'content': self.content,
            'chunk_index': self.chunk_index,
            'word_count': self.word_count
        }

@dataclass
class SimpleQuality:
    """簡化的品質指標"""
    readability: float
    completeness: float
    overall: float
    
    def to_dict(self) -> Dict:
        """轉換為字典格式"""
        return {
            'readability': self.readability,
            'completeness': self.completeness,
            'overall': self.overall
        }

print("✅ 資料結構定義完成")

## 📝 解答 1: SimpleMetadataExtractor 實作

### 設計要點
1. **唯一 ID 生成**: 使用檔案路徑 + 建立時間的 MD5 雜湊
2. **文檔類型分類**: 基於內容關鍵字的啟發式分類
3. **錯誤處理**: 優雅處理檔案讀取異常
4. **可擴展性**: 易於添加新的分類規則

In [ ]:
class SimpleMetadataExtractor:
    """簡單的元資料提取器 - 完整實作"""
    
    def extract_from_file(self, file_path: str, content: str) -> SimpleMetadata:
        """
        從文件和內容提取元資料
        
        Args:
            file_path: 檔案路徑
            content: 檔案內容
            
        Returns:
            SimpleMetadata: 提取的元資料
        """
        file_path = Path(file_path)
        
        try:
            # 獲取檔案統計資訊
            file_stats = file_path.stat()
            
            # 生成唯一文檔 ID (檔案路徑 + 建立時間的 MD5)
            id_string = f"{file_path.name}_{file_stats.st_ctime}"
            doc_id = hashlib.md5(id_string.encode()).hexdigest()[:12]
            
            # 計算字數
            word_count = len(content.split()) if content.strip() else 0
            
            # 分類文檔類型
            doc_type = self._classify_document_type(str(file_path), content)
            
            return SimpleMetadata(
                doc_id=doc_id,
                filename=file_path.name,
                file_size=file_stats.st_size,
                created_time=datetime.fromtimestamp(file_stats.st_ctime),
                doc_type=doc_type,
                word_count=word_count,
                language='en'  # 簡化實作，固定為英文
            )
            
        except Exception as e:
            print(f"⚠️  元資料提取失敗: {e}")
            # 回傳最小化的元資料
            return SimpleMetadata(
                doc_id="unknown",
                filename=file_path.name,
                file_size=0,
                created_time=datetime.now(),
                doc_type="unknown",
                word_count=0
            )
    
    def _classify_document_type(self, file_path: str, content: str) -> str:
        """
        根據路徑和內容分類文檔類型
        
        分類邏輯:
        1. 優先檢查檔案路徑中的關鍵字
        2. 然後檢查內容中的結構關鍵字
        3. 最後根據內容特徵進行啟發式判斷
        """
        file_path_lower = file_path.lower()
        content_lower = content.lower() if content else ""
        
        # 1. 基於檔案路徑的分類
        path_indicators = {
            'paper': ['paper', 'arxiv', 'research', 'journal', 'conference'],
            'report': ['report', 'quarterly', 'annual', 'summary', 'analysis'],
            'policy': ['policy', 'guideline', 'standard', 'procedure']
        }
        
        for doc_type, keywords in path_indicators.items():
            if any(keyword in file_path_lower for keyword in keywords):
                return doc_type
        
        # 2. 基於內容結構的分類
        structure_indicators = {
            'paper': ['abstract', 'introduction', 'methodology', 'conclusion', 
                     'references', 'related work', 'experiments'],
            'report': ['executive summary', 'financial performance', 'quarterly',
                      'recommendations', 'market analysis', 'revenue'],
            'policy': ['policy', 'guidelines', 'procedures', 'compliance',
                      'regulations', 'standards']
        }
        
        # 計算每種類型的匹配分數
        type_scores = {}
        for doc_type, keywords in structure_indicators.items():
            score = sum(1 for keyword in keywords if keyword in content_lower)
            type_scores[doc_type] = score
        
        # 回傳最高分數的類型
        if type_scores and max(type_scores.values()) > 0:
            return max(type_scores, key=type_scores.get)
        
        # 3. 預設分類
        return 'document'

print("✅ SimpleMetadataExtractor 實作完成")

## 📝 解答 2: BasicChunker 實作

### 設計要點
1. **彈性分塊策略**: 支援段落和句子兩種分割方式
2. **大小控制**: 嚴格控制分塊的最小和最大字數
3. **內容過濾**: 自動過濾空白和過短的分塊
4. **ID 管理**: 確保分塊 ID 的唯一性和可追溯性

In [ ]:
class BasicChunker:
    """基礎文檔分塊器 - 完整實作"""
    
    def __init__(self, min_words: int = 10, max_words: int = 300):
        self.min_words = min_words
        self.max_words = max_words
    
    def chunk_by_paragraph(self, content: str, doc_id: str) -> List[SimpleChunk]:
        """
        按段落分塊
        
        演算法步驟:
        1. 以雙換行符分割段落
        2. 清理和過濾段落
        3. 驗證分塊大小
        4. 生成分塊物件
        """
        if not content.strip():
            return []
        
        # 1. 按段落分割
        paragraphs = content.split('\n\n')
        
        # 2. 清理段落並過濾空白
        cleaned_paragraphs = []
        for para in paragraphs:
            cleaned = para.strip()
            if cleaned:  # 過濾空段落
                cleaned_paragraphs.append(cleaned)
        
        # 3. 生成符合大小要求的分塊
        chunks = []
        chunk_index = 0
        
        for para in cleaned_paragraphs:
            # 檢查段落是否符合大小要求
            if self._is_valid_chunk(para):
                chunk = SimpleChunk(
                    chunk_id=f"{doc_id}_chunk_{chunk_index}",
                    doc_id=doc_id,
                    content=para,
                    chunk_index=chunk_index,
                    word_count=len(para.split())
                )
                chunks.append(chunk)
                chunk_index += 1
            
            elif len(para.split()) > self.max_words:
                # 如果段落太長，進一步分割
                sub_chunks = self._split_long_paragraph(para, doc_id, chunk_index)
                chunks.extend(sub_chunks)
                chunk_index += len(sub_chunks)
        
        return chunks
    
    def chunk_by_sentences(self, content: str, doc_id: str, sentences_per_chunk: int = 3) -> List[SimpleChunk]:
        """
        按句子分塊
        
        演算法步驟:
        1. 使用正則表達式分割句子
        2. 清理句子並過濾空白
        3. 將多個句子組合成分塊
        4. 驗證分塊大小並調整
        """
        if not content.strip():
            return []
        
        # 1. 分割句子（基於標點符號）
        sentence_pattern = r'[.!?]+\s+'
        sentences = re.split(sentence_pattern, content)
        
        # 2. 清理句子並過濾空白
        cleaned_sentences = []
        for sent in sentences:
            cleaned = sent.strip()
            if cleaned and len(cleaned.split()) >= 3:  # 至少3個字
                cleaned_sentences.append(cleaned)
        
        # 3. 組合句子成分塊
        chunks = []
        chunk_index = 0
        
        i = 0
        while i < len(cleaned_sentences):
            # 收集指定數量的句子
            chunk_sentences = []
            words_count = 0
            
            # 動態調整句子數量以符合字數限制
            for j in range(sentences_per_chunk):
                if i + j >= len(cleaned_sentences):
                    break
                
                sentence = cleaned_sentences[i + j]
                sentence_words = len(sentence.split())
                
                # 檢查是否超過最大字數
                if words_count + sentence_words > self.max_words and chunk_sentences:
                    break
                
                chunk_sentences.append(sentence)
                words_count += sentence_words
                
                # 如果已達最小字數要求，可以結束這個分塊
                if words_count >= self.min_words:
                    break
            
            if chunk_sentences:
                chunk_content = ' '.join(chunk_sentences)
                
                if self._is_valid_chunk(chunk_content):
                    chunk = SimpleChunk(
                        chunk_id=f"{doc_id}_chunk_{chunk_index}",
                        doc_id=doc_id,
                        content=chunk_content,
                        chunk_index=chunk_index,
                        word_count=words_count
                    )
                    chunks.append(chunk)
                    chunk_index += 1
                
                # 移動到下一組句子
                i += len(chunk_sentences)
            else:
                i += 1
        
        return chunks
    
    def _is_valid_chunk(self, text: str) -> bool:
        """
        檢查分塊是否符合大小要求
        """
        if not text.strip():
            return False
        
        word_count = len(text.split())
        return self.min_words <= word_count <= self.max_words
    
    def _split_long_paragraph(self, paragraph: str, doc_id: str, start_index: int) -> List[SimpleChunk]:
        """
        分割過長的段落
        
        策略: 嘗試在句子邊界分割，避免破壞語義完整性
        """
        # 先嘗試按句子分割
        temp_chunker = BasicChunker(self.min_words, self.max_words)
        sentence_chunks = temp_chunker.chunk_by_sentences(paragraph, doc_id, 2)
        
        # 重新編號
        for i, chunk in enumerate(sentence_chunks):
            chunk.chunk_id = f"{doc_id}_chunk_{start_index + i}"
            chunk.chunk_index = start_index + i
        
        return sentence_chunks

print("✅ BasicChunker 實作完成")

## 📝 解答 3: QualityAssessor 實作

### 設計要點
1. **可讀性評估**: 基於平均句子長度的啟發式算法
2. **完整性評估**: 考慮結構完整性和分塊分佈均勻性
3. **綜合評分**: 加權平均多個維度的分數
4. **穩定性**: 確保邊界條件的處理穩定性

In [ ]:
class QualityAssessor:
    """簡單的品質評估器 - 完整實作"""
    
    def assess_quality(self, content: str, chunks: List[SimpleChunk]) -> SimpleQuality:
        """
        評估文檔品質
        
        評估維度:
        1. 可讀性 (40%) - 基於句子長度分佈
        2. 完整性 (60%) - 基於結構和分塊品質
        """
        if not content.strip():
            return SimpleQuality(0.0, 0.0, 0.0)
        
        # 計算各維度分數
        readability = self._calculate_readability(content)
        completeness = self._calculate_completeness(content, chunks)
        
        # 加權平均計算綜合分數
        weights = {'readability': 0.4, 'completeness': 0.6}
        overall = (
            weights['readability'] * readability +
            weights['completeness'] * completeness
        )
        
        return SimpleQuality(
            readability=readability,
            completeness=completeness,
            overall=overall
        )
    
    def _calculate_readability(self, content: str) -> float:
        """
        計算可讀性分數
        
        算法基礎:
        - 理想句子長度: 15-20 字
        - 分數 = 1.0 - |平均長度 - 理想長度| / 調節因子
        - 考慮句子長度變異性
        """
        try:
            # 分割句子
            sentences = re.split(r'[.!?]+', content)
            sentences = [s.strip() for s in sentences if s.strip()]
            
            if not sentences:
                return 0.0
            
            # 計算句子長度統計
            sentence_lengths = [len(sentence.split()) for sentence in sentences]
            avg_length = np.mean(sentence_lengths)
            std_length = np.std(sentence_lengths)
            
            # 理想長度範圍 (15-20 字)
            ideal_length = 17.5
            length_deviation = abs(avg_length - ideal_length)
            
            # 基礎可讀性分數
            base_score = max(0.0, 1.0 - length_deviation / 25.0)
            
            # 變異性懲罰 (過大的變異性降低可讀性)
            if avg_length > 0:
                cv = std_length / avg_length  # 變異係數
                variability_penalty = min(cv / 2.0, 0.3)  # 最大懲罰30%
            else:
                variability_penalty = 0.0
            
            final_score = max(0.0, base_score - variability_penalty)
            return min(final_score, 1.0)
            
        except Exception:
            return 0.5  # 預設中等可讀性
    
    def _calculate_completeness(self, content: str, chunks: List[SimpleChunk]) -> float:
        """
        計算完整性分數
        
        評估因素:
        1. 結構完整性 (40%) - 是否包含標準文檔結構
        2. 分塊品質 (35%) - 分塊大小分佈的均勻性
        3. 內容長度適中性 (25%) - 內容長度是否合理
        """
        # 1. 結構完整性評估
        structure_score = self._assess_structure_quality(content)
        
        # 2. 分塊品質評估
        chunk_quality_score = self._assess_chunk_quality(chunks)
        
        # 3. 內容長度評估
        length_score = self._assess_content_length(content)
        
        # 加權平均
        completeness = (
            0.4 * structure_score +
            0.35 * chunk_quality_score +
            0.25 * length_score
        )
        
        return min(completeness, 1.0)
    
    def _assess_structure_quality(self, content: str) -> float:
        """
        評估文檔結構品質
        """
        structure_indicators = {
            # 學術論文結構
            'academic': ['abstract', 'introduction', 'method', 'conclusion', 
                        'references', 'background', 'related work'],
            # 報告結構
            'business': ['summary', 'overview', 'analysis', 'recommendations', 
                        'performance', 'results'],
            # 基本結構
            'basic': ['title', 'introduction', 'conclusion', 'summary']
        }
        
        content_lower = content.lower()
        max_score = 0.0
        
        for struct_type, keywords in structure_indicators.items():
            matches = sum(1 for keyword in keywords if keyword in content_lower)
            score = min(matches / len(keywords), 1.0)
            max_score = max(max_score, score)
        
        # 額外加分：包含編號結構 (1. 2. 3.)
        numbered_sections = len(re.findall(r'\n\s*\d+\.\s+', content))
        if numbered_sections >= 2:
            max_score += 0.2
        
        return min(max_score, 1.0)
    
    def _assess_chunk_quality(self, chunks: List[SimpleChunk]) -> float:
        """
        評估分塊品質
        """
        if not chunks:
            return 0.0
        
        # 分塊數量合理性
        chunk_count = len(chunks)
        if chunk_count == 1:
            count_score = 0.6  # 單一分塊扣分
        elif 2 <= chunk_count <= 10:
            count_score = 1.0  # 理想範圍
        elif 11 <= chunk_count <= 20:
            count_score = 0.8  # 稍多但可接受
        else:
            count_score = 0.5  # 過多分塊
        
        # 分塊大小分佈均勻性
        chunk_sizes = [chunk.word_count for chunk in chunks]
        if len(chunk_sizes) > 1:
            mean_size = np.mean(chunk_sizes)
            std_size = np.std(chunk_sizes)
            
            if mean_size > 0:
                cv = std_size / mean_size  # 變異係數
                uniformity_score = max(0.0, 1.0 - cv)  # 變異係數越小越好
            else:
                uniformity_score = 0.0
        else:
            uniformity_score = 1.0
        
        return 0.6 * count_score + 0.4 * uniformity_score
    
    def _assess_content_length(self, content: str) -> float:
        """
        評估內容長度合理性
        
        理想長度範圍:
        - 過短 (<50字): 0.3
        - 短 (50-200字): 0.6  
        - 適中 (200-2000字): 1.0
        - 長 (2000-5000字): 0.8
        - 過長 (>5000字): 0.6
        """
        word_count = len(content.split())
        
        if word_count < 50:
            return 0.3
        elif word_count < 200:
            return 0.6
        elif word_count < 2000:
            return 1.0
        elif word_count < 5000:
            return 0.8
        else:
            return 0.6

print("✅ QualityAssessor 實作完成")

## 📝 解答 4: SimpleDocumentProcessor 整合實作

### 設計要點
1. **統一介面**: 提供簡單易用的 API
2. **錯誤處理**: 優雅處理各種異常情況
3. **批次處理**: 支援目錄級別的批次操作
4. **結果標準化**: 統一的返回格式

In [ ]:
class SimpleDocumentProcessor:
    """簡單的文檔處理器 - 完整實作"""
    
    def __init__(self, chunk_strategy: str = 'paragraph', min_words: int = 10, max_words: int = 300):
        """
        初始化處理器
        
        Args:
            chunk_strategy: 分塊策略 ('paragraph' 或 'sentence')
            min_words: 最小字數
            max_words: 最大字數
        """
        self.chunk_strategy = chunk_strategy
        self.metadata_extractor = SimpleMetadataExtractor()
        self.chunker = BasicChunker(min_words, max_words)
        self.quality_assessor = QualityAssessor()
        
        print(f"✅ 文檔處理器初始化完成 (策略: {chunk_strategy})")
    
    def process_file(self, file_path: str) -> Dict:
        """
        處理單一文件
        
        處理流程:
        1. 讀取檔案內容
        2. 提取元資料
        3. 執行分塊
        4. 評估品質
        5. 回傳結果字典
        
        Returns:
            Dict: 包含 metadata, chunks, quality 的處理結果
        """
        try:
            print(f"📄 處理檔案: {Path(file_path).name}")
            
            # 1. 讀取檔案內容
            content = self._read_file_content(file_path)
            
            # 2. 提取元資料
            metadata = self.metadata_extractor.extract_from_file(file_path, content)
            
            # 3. 執行分塊
            chunks = self._perform_chunking(content, metadata.doc_id)
            
            # 4. 評估品質
            quality = self.quality_assessor.assess_quality(content, chunks)
            
            # 5. 組織回傳結果
            result = {
                'metadata': metadata,
                'chunks': chunks,
                'quality': quality,
                'processing_status': 'success',
                'error_message': None
            }
            
            print(f"✅ 處理完成 - 品質: {quality.overall:.3f}, 分塊: {len(chunks)}")
            return result
            
        except Exception as e:
            print(f"❌ 處理失敗: {e}")
            return {
                'metadata': None,
                'chunks': [],
                'quality': SimpleQuality(0.0, 0.0, 0.0),
                'processing_status': 'failed',
                'error_message': str(e)
            }
    
    def process_directory(self, dir_path: str, file_extensions: List[str] = ['.txt', '.md']) -> List[Dict]:
        """
        處理目錄中的所有文本檔案
        
        Args:
            dir_path: 目錄路徑
            file_extensions: 要處理的檔案副檔名列表
            
        Returns:
            List[Dict]: 所有文件的處理結果
        """
        directory = Path(dir_path)
        
        if not directory.exists() or not directory.is_dir():
            print(f"❌ 目錄不存在或無效: {dir_path}")
            return []
        
        # 收集要處理的檔案
        files_to_process = []
        for ext in file_extensions:
            pattern = f"*{ext}"
            files_to_process.extend(directory.glob(pattern))
        
        print(f"🔍 找到 {len(files_to_process)} 個檔案待處理")
        
        # 批次處理檔案
        results = []
        success_count = 0
        
        for file_path in files_to_process:
            result = self.process_file(str(file_path))
            results.append(result)
            
            if result['processing_status'] == 'success':
                success_count += 1
        
        print(f"\n📊 批次處理完成: {success_count}/{len(files_to_process)} 成功")
        return results
    
    def _read_file_content(self, file_path: str) -> str:
        """
        讀取檔案內容，支援多種編碼
        """
        encodings = ['utf-8', 'utf-8-sig', 'latin-1', 'cp1252']
        
        for encoding in encodings:
            try:
                with open(file_path, 'r', encoding=encoding) as f:
                    return f.read()
            except UnicodeDecodeError:
                continue
        
        raise ValueError(f"無法讀取檔案 {file_path}，嘗試了多種編碼均失敗")
    
    def _perform_chunking(self, content: str, doc_id: str) -> List[SimpleChunk]:
        """
        根據設定的策略執行分塊
        """
        if self.chunk_strategy == 'paragraph':
            return self.chunker.chunk_by_paragraph(content, doc_id)
        elif self.chunk_strategy == 'sentence':
            return self.chunker.chunk_by_sentences(content, doc_id)
        else:
            print(f"⚠️  未知的分塊策略: {self.chunk_strategy}，使用預設段落分塊")
            return self.chunker.chunk_by_paragraph(content, doc_id)
    
    def generate_summary_report(self, results: List[Dict]) -> pd.DataFrame:
        """
        生成處理結果摘要報告
        """
        if not results:
            return pd.DataFrame()
        
        report_data = []
        for result in results:
            if result['processing_status'] == 'success':
                metadata = result['metadata']
                chunks = result['chunks']
                quality = result['quality']
                
                report_data.append({
                    '檔名': metadata.filename,
                    '類型': metadata.doc_type,
                    '字數': metadata.word_count,
                    '檔案大小(KB)': round(metadata.file_size / 1024, 2),
                    '分塊數': len(chunks),
                    '可讀性': round(quality.readability, 3),
                    '完整性': round(quality.completeness, 3),
                    '品質': round(quality.overall, 3),
                    '狀態': '成功'
                })
            else:
                # 處理失敗的檔案
                report_data.append({
                    '檔名': '未知',
                    '類型': '未知',
                    '字數': 0,
                    '檔案大小(KB)': 0,
                    '分塊數': 0,
                    '可讀性': 0.0,
                    '完整性': 0.0,
                    '品質': 0.0,
                    '狀態': '失敗'
                })
        
        return pd.DataFrame(report_data)

print("✅ SimpleDocumentProcessor 整合完成")

## 🧪 完整測試與驗證

### 建立測試資料

In [ ]:
# 建立詳細的測試檔案
test_files = {
    'academic_paper.txt': """
Transformer Networks for Natural Language Processing

Abstract
This paper introduces the Transformer architecture, a novel neural network design based entirely on attention mechanisms. We demonstrate that Transformers achieve state-of-the-art results on machine translation tasks while being more parallelizable than recurrent architectures.

1. Introduction
Natural language processing has been dominated by recurrent neural networks and convolutional neural networks. These architectures have several limitations including sequential processing requirements and difficulty in capturing long-range dependencies.

The Transformer addresses these issues through a self-attention mechanism that relates different positions of a single sequence. This allows for significantly more parallelization and better performance on long sequences.

2. Related Work
Attention mechanisms have been used successfully in various NLP tasks. However, most previous work used attention in conjunction with recurrent or convolutional networks. Our work is the first to rely entirely on attention.

3. Methodology
The Transformer consists of an encoder and decoder, each composed of identical layers. Each layer has two sub-layers: a multi-head self-attention mechanism and a position-wise fully connected feed-forward network.

3.1 Multi-Head Attention
Multi-head attention allows the model to jointly attend to information from different representation subspaces. We compute attention as scaled dot-product attention.

3.2 Position Encoding
Since our model contains no recurrence or convolution, we add positional encodings to the input embeddings to provide information about the relative or absolute position of tokens.

4. Experiments
We evaluate our model on English-German and English-French translation tasks from the WMT 2014 dataset. Our model achieves 28.4 BLEU on English-German translation, establishing a new state-of-the-art.

5. Conclusion
We presented the Transformer, a novel architecture based solely on attention mechanisms. Our results show that attention-based models can outperform recurrent and convolutional networks while being more parallelizable.

References
[1] Vaswani et al. Attention Is All You Need. NIPS 2017.
""",
    
    'business_report.txt': """
Q3 2024 Financial Performance Report

Executive Summary
Our company delivered strong financial results in Q3 2024, with revenue growth of 18% year-over-year. Net income increased to $2.1M, representing a 25% improvement from Q3 2023.

Revenue Analysis
Total revenue reached $15.8M in Q3 2024, compared to $13.4M in the previous year. This growth was driven primarily by increased demand for our cloud services and successful product launches.

Product revenue accounted for 65% of total revenue, while services contributed 35%. The shift toward higher-margin services continued this quarter.

Operating Expenses
Operating expenses were $11.2M, up from $10.1M in Q3 2023. The increase was primarily due to expanded R&D investments and increased headcount to support growth.

Research and development expenses increased to $3.5M, reflecting our commitment to innovation. Sales and marketing expenses were $4.2M, supporting our market expansion efforts.

Market Performance
Customer satisfaction scores remained high at 4.3/5.0. Customer retention rate improved to 92%, up from 89% in the previous quarter. These metrics demonstrate the quality of our products and services.

Our market share in the cloud services segment increased to 12%, positioning us as a strong competitor in this growing market.

Outlook and Recommendations
Looking ahead to Q4 2024, we expect continued growth driven by seasonal demand and new product releases. We recommend increasing investment in customer support to maintain satisfaction levels.

Key initiatives for the next quarter include expanding our international presence and enhancing our product portfolio through strategic partnerships.
""",
    
    'short_note.txt': """
Meeting Notes - Project Kickoff

Date: November 11, 2024
Attendees: John, Sarah, Mike, Lisa

Key Decisions:
- Project timeline: 6 months
- Budget approved: $50K
- Weekly meetings on Wednesdays

Next Steps:
- Sarah will create project plan by Friday
- Mike will set up development environment
- Lisa will coordinate with stakeholders
""",
    
    'technical_doc.txt': """
API Documentation - User Management Service

Overview
The User Management Service provides REST API endpoints for user authentication, authorization, and profile management.

Authentication Endpoints

POST /auth/login
Authenticates a user with email and password. Returns JWT token on success.

Request Body:
{
  "email": "user@example.com",
  "password": "secure_password"
}

Response:
{
  "token": "jwt_token_here",
  "expires_at": "2024-12-11T10:00:00Z"
}

POST /auth/logout
Invalidates the current user session. Requires valid JWT token in Authorization header.

User Profile Endpoints

GET /users/profile
Returns the current user's profile information. Requires authentication.

PUT /users/profile
Updates user profile information. Requires authentication and valid data.

Error Handling
All endpoints return standard HTTP status codes. Error responses include detailed error messages in JSON format.

Rate Limiting
API calls are limited to 1000 requests per hour per user. Exceeding this limit returns HTTP 429 status.
"""
}

# 建立測試目錄和檔案
test_dir = SOLUTION_DIR / 'test_documents'
test_dir.mkdir(parents=True, exist_ok=True)

for filename, content in test_files.items():
    file_path = test_dir / filename
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(content)

print(f"✅ 建立了 {len(test_files)} 個測試檔案於 {test_dir}")

### 執行完整測試流程

In [ ]:
print("🧪 開始完整測試流程\n")

# 測試不同分塊策略
strategies = ['paragraph', 'sentence']
all_results = {}

for strategy in strategies:
    print(f"\n📋 測試策略: {strategy.upper()}")
    print("=" * 50)
    
    # 初始化處理器
    processor = SimpleDocumentProcessor(
        chunk_strategy=strategy,
        min_words=15,
        max_words=200
    )
    
    # 批次處理
    results = processor.process_directory(str(test_dir))
    all_results[strategy] = results
    
    # 生成報告
    report_df = processor.generate_summary_report(results)
    print(f"\n📊 {strategy.upper()} 策略處理報告:")
    print(report_df.to_string(index=False))
    
    # 統計分析
    successful_results = [r for r in results if r['processing_status'] == 'success']
    if successful_results:
        avg_quality = np.mean([r['quality'].overall for r in successful_results])
        avg_chunks = np.mean([len(r['chunks']) for r in successful_results])
        
        print(f"\n📈 {strategy.upper()} 策略統計:")
        print(f"平均品質分數: {avg_quality:.3f}")
        print(f"平均分塊數量: {avg_chunks:.1f}")
        print(f"處理成功率: {len(successful_results)}/{len(results)} ({len(successful_results)/len(results)*100:.1f}%)")

print("\n" + "=" * 60)
print("🎉 完整測試流程結束！")

### 詳細結果分析

In [ ]:
print("🔍 詳細結果分析\n")

# 分析單一檔案的處理結果
sample_file = test_dir / 'academic_paper.txt'
processor_detailed = SimpleDocumentProcessor(chunk_strategy='paragraph')
detailed_result = processor_detailed.process_file(str(sample_file))

if detailed_result['processing_status'] == 'success':
    metadata = detailed_result['metadata']
    chunks = detailed_result['chunks']
    quality = detailed_result['quality']
    
    print(f"📋 檔案: {metadata.filename}")
    print(f"文檔 ID: {metadata.doc_id}")
    print(f"文檔類型: {metadata.doc_type}")
    print(f"總字數: {metadata.word_count}")
    print(f"檔案大小: {metadata.file_size} bytes")
    
    print(f"\n📊 品質評估詳細:")
    print(f"可讀性分數: {quality.readability:.3f}")
    print(f"完整性分數: {quality.completeness:.3f}")
    print(f"綜合品質: {quality.overall:.3f}")
    
    print(f"\n📝 分塊詳細資訊:")
    print(f"總分塊數: {len(chunks)}")
    
    for i, chunk in enumerate(chunks[:3]):  # 顯示前3個分塊
        print(f"\n分塊 {i+1}:")
        print(f"  ID: {chunk.chunk_id}")
        print(f"  字數: {chunk.word_count}")
        print(f"  內容預覽: {chunk.content[:150]}...")
    
    if len(chunks) > 3:
        print(f"\n... (還有 {len(chunks) - 3} 個分塊)")
    
    # 分塊大小統計
    chunk_sizes = [chunk.word_count for chunk in chunks]
    print(f"\n📈 分塊統計:")
    print(f"最小分塊: {min(chunk_sizes)} 字")
    print(f"最大分塊: {max(chunk_sizes)} 字")
    print(f"平均分塊: {np.mean(chunk_sizes):.1f} 字")
    print(f"標準差: {np.std(chunk_sizes):.1f}")

else:
    print(f"❌ 處理失敗: {detailed_result['error_message']}")

## 🎓 解答總結與學習重點

### 🏆 實作亮點

1. **模組化設計**
   - 每個組件職責單一明確
   - 易於測試和維護
   - 支援靈活配置

2. **錯誤處理機制**
   - 優雅處理檔案讀取異常
   - 多種編碼格式支援
   - 邊界條件保護

3. **演算法設計**
   - 啟發式文檔分類
   - 動態分塊大小調整
   - 多維度品質評估

### 🔑 關鍵學習點

#### 1. 元資料提取策略
```python
# 組合多種資訊來源
- 檔案系統資訊 (大小、時間)
- 檔案路徑特徵
- 內容結構分析
- 啟發式分類規則
```

#### 2. 分塊演算法核心
```python
# 段落分塊: 保持語義完整性
# 句子分塊: 精細控制分塊大小
# 動態調整: 根據內容特徵調整策略
```

#### 3. 品質評估框架
```python
# 多維度評估
- 可讀性: 句子長度分佈
- 完整性: 結構 + 分塊品質 + 長度
- 綜合評分: 加權平均
```

### 🚀 進階改進方向

1. **語義分塊**: 整合 sentence-transformers
2. **多語言支援**: 添加語言檢測和分詞
3. **並發處理**: 實作 async/await 並發
4. **配置管理**: JSON/YAML 配置檔案
5. **結果快取**: 避免重複處理

### 💡 最佳實務總結

1. **始終考慮邊界條件** - 空檔案、超大檔案、特殊字符
2. **提供清晰的錯誤信息** - 幫助使用者快速定位問題
3. **保持介面簡潔** - 隱藏複雜性，提供簡單 API
4. **文檔和測試同等重要** - 確保代碼可維護性
5. **效能與品質並重** - 在速度和準確性之間找平衡

---

## 🎯 下一步學習建議

完成本解答學習後，建議：

1. **嘗試改進算法** - 實作語義分塊、優化品質評估
2. **處理真實資料** - 測試不同格式和語言的文檔
3. **整合進階工具** - 結合 Docling、spaCy 等專業工具
4. **學習模組 2** - 進入元資料管理與索引建立

恭喜完成基礎文檔處理的學習！🎉